In [1]:
import csv
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle


Using TensorFlow backend.


In [2]:
DS = pd.read_csv("RaivoSubset150SentenceLength.csv")

In [3]:
DS['tag'].fillna('O', inplace=True)

In [4]:
words = list(set(DS["word"].values))
n_words = len(words)
n_words

27741

In [5]:
tags = list(set(DS["tag"].values))
n_tags = len(tags); n_tags


2

In [6]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("text_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [7]:
getter = SentenceGetter(DS)

In [8]:
sentences = getter.sentences


In [9]:
sentences[1]

[('Pt.', 'O'),
 ('tehtud', 'O'),
 ('3', 'O'),
 ('plasmaferees,', 'O'),
 ('talus', 'O'),
 ('hästi.', 'O'),
 ('Eemaldatud', 'O'),
 ('700', 'O'),
 ('ml', 'O'),
 ('plasmat.', 'O'),
 ('RR', 'object'),
 ('112/60', 'object'),
 ('mmHg.', 'O'),
 ('Tgasi', 'O'),
 ('22.10.09.', 'O')]

In [10]:
# np.median([len(s) for s in sentences])
DS.word.apply(lambda x: len(x)).describe()

count    157280.000000
mean          6.230697
std           4.330117
min           1.000000
25%           3.000000
50%           5.000000
75%           9.000000
max          54.000000
Name: word, dtype: float64

In [11]:
LengthsList = []
for i in range(len(sentences)):
    LengthsList.append(len(sentences[i]))
np.max(LengthsList)

150

In [12]:
sentences[6]

[('Kokkuvõte:', 'O'),
 ('<ANONYM', 'O'),
 ('id="0"', 'O'),
 ('type="per"', 'O'),
 ('morph="_A_', 'O'),
 ('sg', 'O'),
 ('n"/>', 'O'),
 ('nähtava', 'O'),
 ('patoloogiata.', 'O'),
 ('Kokkuvõte:', 'O'),
 ('<ANONYM', 'O'),
 ('id="0"', 'O'),
 ('type="per"', 'O'),
 ('morph="_A_', 'O'),
 ('sg', 'O'),
 ('n"/>', 'O'),
 ('nähtava', 'O'),
 ('patoloogiata.', 'O'),
 ('Kokkuvõte:', 'O'),
 ('<ANONYM', 'O'),
 ('id="0"', 'O'),
 ('type="per"', 'O'),
 ('morph="_A_', 'O'),
 ('sg', 'O'),
 ('n"/>', 'O'),
 ('nähtava', 'O'),
 ('patoloogiata.', 'O'),
 ('Kokkuvõte:', 'O'),
 ('<ANONYM', 'O'),
 ('id="0"', 'O'),
 ('type="per"', 'O'),
 ('morph="_A_', 'O'),
 ('sg', 'O'),
 ('n"/>', 'O'),
 ('nähtava', 'O'),
 ('patoloogiata.', 'O'),
 ('Kokkuvõte:', 'O'),
 ('<ANONYM', 'O'),
 ('id="0"', 'O'),
 ('type="per"', 'O'),
 ('morph="_A_', 'O'),
 ('sg', 'O'),
 ('n"/>', 'O'),
 ('nähtava', 'O'),
 ('patoloogiata.', 'O'),
 ('Kokkuvõte:', 'O'),
 ('<ANONYM', 'O'),
 ('id="0"', 'O'),
 ('type="per"', 'O'),
 ('morph="_A_', 'O'),
 ('sg', 'O')

In [14]:
max_len = 150
max_len_char = 10

In [15]:
tag2idx = {t: i + 1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0
idx2tag = {i: w for w, i in tag2idx.items()}

In [16]:
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)


104


In [17]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1 #For predictions
char2idx["PAD"] = 0

In [18]:
idx2char = {i: w for w, i in char2idx.items()}

In [19]:
X_char = []
for sentence in sentences:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))

In [20]:
y = [[tag2idx[w[1]] for w  in s] for s in sentences]


In [21]:
len(sentences)

13643

In [22]:
y = pad_sequences(maxlen=max_len, sequences=y, value=tag2idx["PAD"], padding='post', truncating='post')


In [23]:
from sklearn.model_selection import train_test_split
X_char_tr, X_char_te, y_tr,  y_te= train_test_split(X_char, y, test_size=0.2, random_state=2018)


In [24]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D


# input and embeddings for characters
char_in = Input(shape=(max_len, max_len_char,))
emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=10,
                           input_length=max_len_char, mask_zero=True))(char_in)
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                                recurrent_dropout=0.5))(emb_char)

# main LSTM
x = SpatialDropout1D(0.3)(char_enc)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=True,
                               recurrent_dropout=0.6))(x)
out = TimeDistributed(Dense(n_tags + 1, activation="softmax"))(main_lstm)

model = Model( char_in, out)




In [25]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 10)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 150, 10, 10)       1060      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 150, 20)           2480      
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 150, 20)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 150, 100)          28400     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 150, 3)            303       
Total params: 32,243
Trainable params: 32,243
Non-trainable params: 0
_______________________________________________________

In [ ]:
history = model.fit(np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char)),
                    np.array(y_tr).reshape(len(y_tr), max_len, 1),
                    batch_size=32, epochs=10, validation_split=0.1, verbose=1)


C:\Users\Kasutaja\Anaconda3\envs\myEnv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 9822 samples, validate on 1092 samples
Epoch 1/10
9822/9822 [==============================] - 74s 7ms/step - loss: 0.0222 - acc: 0.9227 - val_loss: 0.0060 - val_acc: 0.9822
Epoch 2/10
9822/9822 [==============================] - 75s 8ms/step - loss: 0.0054 - acc: 0.9836 - val_loss: 0.0044 - val_acc: 0.9850
Epoch 3/10
9822/9822 [==============================] - 75s 8ms/step - loss: 0.0044 - acc: 0.9852 - val_loss: 0.0033 - val_acc: 0.9868
Epoch 4/10
9822/9822 [==============================] - 74s 7ms/step - loss: 0.0036 - acc: 0.9874 - val_loss: 0.0026 - val_acc: 0.9902
Epoch 5/10
9822/9822 [==============================] - 73s 7ms/step - loss: 0.0031 - acc: 0.9894 - val_loss: 0.0021 - val_acc: 0.9923
Epoch 6/10
5504/9822 [===============>..............] - ETA: 30s - loss: 0.0029 - acc: 0.9900

In [ ]:
hist = pd.DataFrame(history.history)


In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure(figsize=(12,12))
plt.plot(hist["acc"])
plt.plot(hist["val_acc"])
plt.show()


In [ ]:
y_pred = model.predict(
                        np.array(X_char_te).reshape((len(X_char_te),
                                                     max_len, max_len_char)))


In [ ]:
def charindx2word(Tword):
    word = []
    for i in range(len(Tword)):
        if(Tword[i] !=0):
            word.append(idx2char[Tword[i]])
    return "".join(word)

In [ ]:
i = 1000
p = np.argmax(y_pred[i], axis=-1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_char_te[i], y_te[i], p):
    if t != 0: #PAD
        print("{:15}: {:5} {}".format(charindx2word(w), idx2tag[t], idx2tag[pred]))


In [ ]:
i = 14
p = np.argmax(y_pred[i], axis=-1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_char_te[i], y_te[i], p):
    if t != 0: #PAD
        print("{:15}: {:5} {}".format(charindx2word(w), idx2tag[t], idx2tag[pred]))


In [ ]:
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p, axis=-1)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
pred_labels = pred2label(y_pred)

def pred2labelV2(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p])
        out.append(out_i)
    return out
test_labels = pred2labelV2(y_te)

In [ ]:
from seqeval.metrics import accuracy_score,precision_score, recall_score, f1_score, classification_report
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

In [ ]:
from sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

In [ ]:
print(classification_report(test_labels,pred_labels))